In [1]:
%pip install imageio imageio-ffmpeg pygame numerize casadi stable-baselines3 tensorboard "stable-baselines3[extra]"  pyvirtualdisplay ipywidgets --quiet
%pip install "gymnasium[other]" --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import copy
import gymnasium as gym
import stable_baselines3
from SimulationConfigLoader import VehicleConfigLoader, MapConfigLoader
from Simulation import ArticulatedVehicle
from Simulation import Map
from Simulation import MapEntity
import random
import Visualization
from casadi import cos, sin, tan
from typing import Any, SupportsFloat
from stable_baselines3 import PPO, SAC
import torch
import os
from IPython.display import HTML, display
from numerize import numerize
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import platform
from IPython.display import clear_output

/home/walter/Área de trabalho/deep_learning_final_project/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
log_dir = "logs"

#### API Gymnasium

Aqui é definido o ambiente de treinamento, extendendo a classe gym.Env.

In [4]:
class ParkingEnv(gym.Env):

    metadata = {"render_modes": ["rgb_array"], "render_fps": 24}


    ## ambiente
    VEHICLE_NAME = "BUG1" #nome do veículo
    MAP_NAME = "MAPA_1" #nome do mapa
    SENSOR_RANGE_M = 50.0 # raio do sensor
    SPEED_LIMIT_MS = 5.0 # velocidade maxima
    STEERING_LIMIT_RAD = float(np.deg2rad(28.0)) # angulo maximo de esterçamento
    JACKKNIFE_LIMIT_RAD = float(np.deg2rad(65.0)) # angulo maximo de jackknife
    DT = 0.5 # tempo de simulação
    MAX_SECONDS = 120.0
    MAX_STEPS = int(MAX_SECONDS / DT)


    ## recompensas
    MAX_PUNISHMENT_TIME_PER_EPISODE = -20.0 # penalidade maxima por tempo acumulada por episodio
    #deve ser menor que a punição por colisão e jackknife, senão o agente vai colidir propositalmente

    REWARD_GOAL = 100.0 # recompensa por chegar ao objetivo
    PUNISHMENT_TIME = MAX_PUNISHMENT_TIME_PER_EPISODE / MAX_STEPS # penalidade por tempo a cada passo
    PUNISHMENT_ZERO_SPEED = 5 * PUNISHMENT_TIME # penalidade por velocidade zero - 5 vezes maior que a penalidade por tempo
    PUNISHMENT_COLLISION = -150.0 # penalidade por colisão
    PUNISHMENT_JACKKNIFE = -150.0 # penalidade por jackknife
    PROGRESS_REWARD_MULTIPLIER = 0.1 # multiplicador da recompensa por progresso (recompensa = metros ganhos * multiplicador)

    def __init__(self, seed = 0):

        self.render_mode = "rgb_array"

        self.vehicle_loader = VehicleConfigLoader("config/lista_veiculos.json")
        self.map_loader = MapConfigLoader("config/lista_mapas.json")

        self.vehicle = self.vehicle_loader.load_vehicle(self.VEHICLE_NAME)
        self.map = self.map_loader.load_map(self.MAP_NAME)

        self.map.place_vehicle(self.vehicle)

        self.steps = 0
        self.total_reward = 0.0

        self.last_distance_to_goal = self._calculate_goal_distance_manhattan()

        
        # Gymnasium spaces based on README specifications


        # Observation: [x, y, theta, beta, alpha, r1..r14, goal_x, goal_y, goal_theta]
        obs_low = np.array(
            [
                -np.pi,                      # theta
                -self.JACKKNIFE_LIMIT_RAD,        # beta
                -self.STEERING_LIMIT_RAD,         # alpha
            ]
            + [0.0] * self.vehicle.get_raycast_count()                     # raycasts
            + [0.0, -np.pi],            # goal distance, goal direction
            dtype=np.float32,
        )

        obs_high = np.array(
            [
                np.pi,                       # theta
                self.JACKKNIFE_LIMIT_RAD,         # beta
                self.STEERING_LIMIT_RAD,          # alpha
            ]
            + [self.SENSOR_RANGE_M] * self.vehicle.get_raycast_count()          # raycasts
            + [100, np.pi], # goal distance, goal direction
            dtype=np.float32,
        )

        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)

        # Action: [v, alpha]
        act_low = np.array(
            [
                -self.SPEED_LIMIT_MS,             # v (allow reverse)
                -self.STEERING_LIMIT_RAD,         # alpha
            ],
            dtype=np.float32,
        )
        act_high = np.array(
            [
                self.SPEED_LIMIT_MS,             # v (allow reverse)
                self.STEERING_LIMIT_RAD,          # alpha
            ],
            dtype=np.float32,
        )
        self.action_space = gym.spaces.Box(low=act_low, high=act_high, dtype=np.float32)

    def reset(self, *, seed: int | None = None, options: dict | None = None):
        super().reset(seed=seed)

        self.vehicle = self.vehicle_loader.load_vehicle(self.VEHICLE_NAME)
        self.map = self.map_loader.load_map(self.MAP_NAME)

        self.map.place_vehicle(self.vehicle)

        self.steps = 0
        self.total_reward = 0.0

        self.last_distance_to_goal = self._calculate_goal_distance_manhattan()

        # Build observation
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()
        alpha_current = self.vehicle.get_alpha()
        raycast_lengths = self.vehicle.get_raycast_lengths()
        goal_distance = self._calculate_goal_distance()
        goal_direction = self._calculate_goal_direction()
        observation = np.array([theta, beta, alpha_current] + raycast_lengths + [goal_distance, goal_direction], dtype=np.float32)
        info = {}
        return observation, info

    def step(self, action) -> tuple[np.ndarray, SupportsFloat, bool, bool, dict[str, Any]]:
        velocity, alpha = action
        self.steps += 1
        terminated = False
        truncated = False
        info = {}
        reward = self.PUNISHMENT_TIME # recompensa base por passo de tempo (reduzida)

        self._move_vehicle(velocity, alpha, self.DT)

        ## Strongly punish zero/low speed
        if abs(velocity) < 0.1:
            reward = self.PUNISHMENT_ZERO_SPEED  # Penalidade muito maior para ficar parado

        if self.steps >= self.MAX_STEPS:
            truncated = True
        if self._check_vehicle_parking():
            terminated = True
            reward = self.REWARD_GOAL
        elif self._check_vehicle_collision():
            terminated = True
            reward = self.PUNISHMENT_COLLISION
        elif self._check_trailer_jackknife():
            terminated = True
            reward = self.PUNISHMENT_JACKKNIFE

        new_distance_to_goal = self._calculate_goal_distance_manhattan()

        # Recompensa baseada no progresso real
        progress_reward = (self.last_distance_to_goal - new_distance_to_goal)
        reward += progress_reward * self.PROGRESS_REWARD_MULTIPLIER # Ajuste o multiplicador (0.1) conforme necessário

        self.last_distance_to_goal = new_distance_to_goal

        # Observação: [x, y, theta, beta, alpha, r1..r14, goal_dist, goal_direction]
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()
        alpha_current = self.vehicle.get_alpha()
        raycast_lengths = self.vehicle.get_raycast_lengths()
        observation = np.array([theta, beta, alpha_current] + raycast_lengths + [self._calculate_goal_distance(), self._calculate_goal_direction()], dtype=np.float32)

        self.total_reward += reward

        return observation, reward, terminated, truncated, info

    def render(self):
        rgb_array = Visualization.to_rgb_array(self.map, self.vehicle, (288, 288), self._calculate_goal_distance(), self._calculate_goal_direction(), total_reward=self.total_reward)
        return rgb_array

    def close(self):
        pass


    def _move_vehicle(self, velocity: float, alpha: float, dt: float):
        # Current state
        x, y = self.vehicle.get_position()
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()

        # Geometry
        D = self.vehicle.get_distancia_eixo_dianteiro_quinta_roda() - self.vehicle.get_distancia_eixo_traseiro_quinta_roda()
        L = self.vehicle.get_distancia_eixo_traseiro_trailer_quinta_roda()
        a = self.vehicle.get_distancia_eixo_traseiro_quinta_roda()

        angular_velocity_tractor = (velocity / D) * tan(alpha)

        # Kinematics
        x_dot = velocity * cos(theta)
        y_dot = velocity * sin(theta)
        theta_dot = (velocity / D) * tan(alpha)
        beta_dot = angular_velocity_tractor * (1 - (a * cos(beta)) / L) - (velocity * sin(beta)) / L

        # Euler step
        new_x = x + x_dot * dt
        new_y = y + y_dot * dt
        new_theta = theta + theta_dot * dt
        new_beta = beta + beta_dot * dt

        self.vehicle.update_physical_properties(new_x, new_y, velocity, new_theta, new_beta, alpha)
        self.vehicle.update_raycasts(self.map.get_entities())

    def _check_vehicle_collision(self) -> bool:
        """Verifica se o veículo colidiu com alguma parede ou passou por cima de uma vaga de estacionamento."""
        for entity in self.map.get_entities():
            if entity.type == MapEntity.ENTITY_WALL or entity.type == MapEntity.ENTITY_PARKING_SLOT:
                if self.vehicle.check_collision(entity):
                    return True
        return False

    def _calculate_goal_distance(self):
        x, y = self.vehicle.get_position()
        goal_x, goal_y = self.map.get_parking_goal_position()
        distance_to_goal = np.sqrt((x - goal_x)**2 + (y - goal_y)**2)
        return distance_to_goal

    def _calculate_goal_distance_manhattan(self):
        x, y = self.vehicle.get_position()
        goal_x, goal_y = self.map.get_parking_goal_position()
        distance_to_goal = abs(x - goal_x) + abs(y - goal_y)
        return distance_to_goal

    def _calculate_goal_direction(self):
        x, y = self.vehicle.get_position()
        goal_x, goal_y = self.map.get_parking_goal_position()
        direction_to_goal = np.arctan2(goal_y - y, goal_x - x)
        return direction_to_goal

    def _check_vehicle_parking(self) -> bool:
        """Verifica se o trailer do veículo está dentro de uma vaga de estacionamento."""
        goal = self.map.get_parking_goal()
        if goal.get_bounding_box().contains_bounding_box(self.vehicle.get_bounding_box_trailer()):
            return True
        return False

    def _check_trailer_jackknife(self) -> bool:
        """Verifica se o trailer do veículo está em jackknife."""
        return self.vehicle.get_beta() > np.deg2rad(65.0) or self.vehicle.get_beta() < np.deg2rad(-65.0)



#### Funções de treinamento e avaliação

In [ ]:
def evaluate_model(model: PPO, iterations: int = 10):
    rewards = []
    for _ in range(iterations):
        rewards.append(run_episode(model, int(random.random() * 1000)))
    return rewards
    
def run_episode_and_save_video(model):
    video_recorder = Visualization.VideoRecorder("simulation.mp4", fps=10)
    env = ParkingEnv()
    observation, info = env.reset()
    total_reward = 0.0

    while(True):
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        video_recorder.append(env.render())
        if terminated or truncated:
            break

    video_recorder.close()
    env.close()
    return total_reward

def run_episode(model, seed = None):
    env = ParkingEnv(seed)
    observation, info = env.reset()
    total_reward = 0.0

    while(True):
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        if terminated or truncated:
            break

    env.close()
    return total_reward


from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import platform

N_ENVS = 8

def make_env(seed: int = 0):
    def _init():
        env = ParkingEnv()
        env = stable_baselines3.common.monitor.Monitor(env, log_dir)
        env.reset(seed=seed)
        return env
    return _init

def make_vector_env(n_envs: int = N_ENVS, use_subproc: bool | None = None):
    """Create a vectorized environment.

    On Windows (especially inside notebooks), `SubprocVecEnv` can be unstable
    and cause BrokenPipeError/EOFError. There we default to `DummyVecEnv`.
    """
    # Auto-select backend if not specified
    if use_subproc is None:
        use_subproc = platform.system() != "Windows"

    env_fns = [make_env(seed=i) for i in range(n_envs)]

    if use_subproc:
        try:
            return SubprocVecEnv(env_fns)
        except Exception as e:
            print(f"SubprocVecEnv failed ({e}), falling back to DummyVecEnv.")

    return DummyVecEnv(env_fns)




def train_sac(model: SAC | None = None, total_timesteps: int = 10000, save_every: int | None = None, save_path: str | None = None, save_name: str | None = None):
    """
        Treina o modelo SAC (Soft Actor-Critic)
        Args:
            model: Modelo SAC a ser treinado, se None, cria um novo modelo, senão treina modelo existente
            total_timesteps: Total de timesteps para treinar
            save_every: O modelo será salvo a cada save_every timesteps durante o treinamento
            save_path: Caminho para o diretório onde o modelo será salvo
            save_name: Nome do modelo
        Returns:
            Modelo SAC treinado
    """
    # Create environment
    env = make_vector_env()

    # Hiperparâmetros da rede neural
    # O README descreve um espaço de observação com 19 dimensões
    # (3 ângulos, 14 raycasts, 2 de meta).
    # Aumentamos a rede para [256, 256] para lidar com essa complexidade.
    # No SAC (Actor-Critic), temos redes 'pi' (policy/ator) e 'qf' (Q-function/crítico).
    policy_kwargs = dict(activation_fn=torch.nn.ReLU,
                         net_arch=dict(pi=[256, 256], qf=[256, 256]))

    # Instancia o modelo SAC usando os hiperparâmetros definidos
    if model is None:
        model = SAC(
            policy="MlpPolicy",          # Arquitetura da política (MLP para observações vetoriais)
            env=env,                     # Ambiente compatível com Gymnasium
            policy_kwargs=policy_kwargs, # Arquitetura de rede customizada
            verbose=0,                   # Verbosidade do log
            tensorboard_log=log_dir,     # Diretório para logs do TensorBoard
            learning_rate=0.0001,        # Taxa de aprendizado (um bom padrão)
            buffer_size=1000000,          # Tamanho do replay buffer (SAC é off-policy)
            batch_size=256,              # Tamanho do batch amostrado do buffer
            gamma=0.9995,                # Fator de desconto (mantido alto, pois estacionar é uma tarefa de horizonte longo)
            ent_coef="auto",             # Coeficiente de entropia (essencial no SAC, 'auto' aprende automaticamente)
            tau=0.005,                   # Coeficiente de "soft update" para as redes de target
            learning_starts=1000,        # Número de passos antes de começar a treinar (coleta experiência)
            device="auto"                # Usa GPU se disponível
        )
    else:
        model.set_env(env)

    timesteps_split = 0
    if save_every is not None:
        timesteps_split = int(total_timesteps / save_every)
        for i in range(timesteps_split):
            model.learn(total_timesteps=save_every, progress_bar=True, tb_log_name=save_name, reset_num_timesteps=False,)
            model_save_dir = os.path.join(save_path, save_name)
            model.save(model_save_dir)
            clear_output(wait=True)

    return model

#### Funções auxiliares para visualização

In [6]:
# @title Play Video function
from IPython.display import HTML
from base64 import b64encode
import platform

# Only import and use pyvirtualdisplay on Linux
if platform.system() != 'Windows':
    from pyvirtualdisplay import Display
else:
    Display = None

# create the directory to store the video(s)
os.makedirs("./video", exist_ok=True)

# Only start virtual display on Linux (not needed on Windows)
display = None
if platform.system() != 'Windows' and Display is not None:
    display = Display(visible=False, size=(2000, 1500))
    _ = display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  if not os.path.exists(videopath):
      return f'<p>Video file not found: {videopath}</p>'
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

def record_and_display_video_manual(env, model, video_name, num_episodes=1):
    """
    Records a video manually using Visualization.VideoRecorder (more reliable).
    
    Args:
        env: The gymnasium environment.
        model: The trained model.
        video_name (str): The name to use for the video file.
        num_episodes (int): The number of episodes to record (default is 1).
    """
    os.makedirs("./video", exist_ok=True)
    
    video_path = f"video/{video_name}.mp4"
    video_recorder = Visualization.VideoRecorder(video_path, fps=10)
    
    total_reward = 0.0
    episode_count = 0
    
    for episode in range(num_episodes):
        observation, info = env.reset()
        episode_reward = 0.0
        
        while(True):
            action, _ = model.predict(observation, deterministic=True)
            observation, reward, terminated, truncated, info = env.step(action)
            episode_reward += float(reward)
            video_recorder.append(env.render())
            
            if terminated or truncated:
                break
        
        total_reward += episode_reward
        episode_count += 1
    
    video_recorder.close()
    print(f"\nTotal reward: {total_reward}")
    print(f"Video saved to: {video_path}")
    
    html = render_mp4(video_path)
    return HTML(html)

def record_and_display_video(env, model, video_name, num_episodes=1):
    """
    Records a video of the agent performing in the environment and displays it.

    Args:
        env: The gymnasium environment.
        model: The trained model.
        video_name (str): The name to use for the video file.
        num_episodes (int): The number of episodes to record (default is 1).
    """
    import glob
    
    # create the directory to store the video(s)
    os.makedirs("./video", exist_ok=True)

    # Use a virtual display for rendering (only on Linux)
    display = None
    if platform.system() != 'Windows' and Display is not None:
        display = Display(visible=False, size=(1400, 900))
        _ = display.start()

    env_name = "ParkingEnv"

    env = gym.wrappers.RecordVideo(
        env,
        video_folder="video",
        name_prefix=f"{env_name}_{video_name}",
        episode_trigger=lambda episode_id: episode_id < num_episodes
    )

    observation, _ = env.reset()
    total_reward = 0
    done = False
    episode_count = 0

    while not done:
        action, states = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward
        if done:
            episode_count += 1
            if episode_count < num_episodes:
                observation, _ = env.reset()
                done = False

    env.close()
    # Stop the virtual display if it was started
    if display is not None:
        display.stop()

    print(f"\nTotal reward: {total_reward}")

    # Find the video file that was created
    video_pattern = f"video/{env_name}_{video_name}*.mp4"
    video_files = glob.glob(video_pattern)
    
    if not video_files:
        # Try alternative pattern
        video_pattern = f"video/*{video_name}*.mp4"
        video_files = glob.glob(video_pattern)
    
    if not video_files:
        # List all video files for debugging
        all_videos = glob.glob("video/*.mp4")
        print(f"Warning: Expected video file not found. Available video files: {all_videos}")
        return HTML("<p>Video file not found. Check the video directory.</p>")
    
    # Use the first matching video file
    video_path = video_files[0]
    print(f"Found video file: {video_path}")
    
    # show video
    html = render_mp4(video_path)
    return HTML(html)

In [7]:
import shutil
def clean_logs():
    if os.path.exists(log_dir):
        print("Cleaning logs")
        shutil.rmtree(log_dir)
    os.makedirs(log_dir, exist_ok=True)



In [8]:
model_name = "SAC_model_16_11"
model_save_dir = "models"
total_training_timesteps = 100000000
save_every = 100000

In [ ]:
os.makedirs(model_save_dir, exist_ok=True)

#se modelo salvo já existe, carrega
if(os.path.exists(os.path.join(model_save_dir, model_name + ".zip"))):
    model_save_path = os.path.join(model_save_dir, model_name + ".zip")
    model = SAC.load(model_save_path)
else: #senão, cria novo
    model = None

model = train_sac(model, total_timesteps=total_training_timesteps, save_every=save_every, save_path=model_save_dir, save_name=model_name)


Output()

#### Carregar modelo já existente

In [ ]:
model_name = "SAC_model_16_11"

model_save_path = os.path.join(model_save_dir, model_name + ".zip")
model = SAC.load(model_save_path)


In [ ]:
env = ParkingEnv()
record_and_display_video_manual(env, model, "ppo_model", num_episodes=1)

Saved 240 frames as MP4 to video/ppo_model.mp4

Total reward: -95.08963355794549
Video saved to: video/ppo_model.mp4


In [ ]:
total_rewards = evaluate_model(model, 100)
total_rewards = np.array(total_rewards)
print(f"mean reward {total_rewards.mean()}")
print(f"std reward {total_rewards.std()}")
print(f"min reward {total_rewards.min()}")
print(f"max reward {total_rewards.max()}")
